<a href="https://colab.research.google.com/github/Moe9811/T.N.O-model/blob/main/WS_V6_T_N_O_model_17_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TOWER Network Integrated Model : Integer Programming Model

## Packages & Libraries

In [42]:
!pip install pulp

In [43]:
import pulp as lp
import numpy as np
import pandas as pd

## Model info:



Product List:


1.   KT 400
2.   KTM 52
3.   KTM 32





Customer list:

1.   AIT Worldwide Logistics
2.   Biocair Inc
3.   DSV
4.   Expeditors International (Puerto Rico) Inc.
5.   Movianto UK Ltd
6.   Quick International Courier Inc
7.   World Courier UK Ltd

Hub list:


1.   Brussels- BRU
2.   Chicago -ORD
3.   Penssauken - PHC
4.   Reading/Theale - THE
5.   Los Angeles - LAD
6.   Frankfurt - FRA
7.  Shanghai - SHA
8.  San Juan - SJU
9.   Amsetrdam - AMS

Mode of Transport:



1.   Road
2.   Air
3.   Sea



## Sets and indices

In [44]:
H = list(range(1, 10))  # Hub indices : 9 unique hubs in Model info
P = list(range(1, 4))   # 3 unique containers in model info
T = list(range(1, 13))  # time periods : 12 months giving Monthly recommendations
I = list(range(1, 8))   # 7 3PL providers
J = list(range(1, 3))   # State of the container (1: Used, 2: Clean)
M = list(range(1,4))    # 3 modes of tranpsort: Road, Air & Sea

## Decsion Variables




In [45]:
# Decision variables
deliver = lp.LpVariable.dict("delivered", (H, I, P, T), lowBound = 0, upBound = None, cat = lp.LpInteger) #Qhi

collect = lp.LpVariable.dict("collected", (I, H, P, T), lowBound = 0, upBound = None, cat = lp.LpInteger) #Rih

move = lp.LpVariable.dict("moved", (H, H, P, J , M, T), lowBound = 0, upBound = None, cat = lp.LpInteger)  #Shh

inventory = lp.LpVariable.dict("inventory", (H, P, J, T), lowBound = 0, upBound = None, cat = lp.LpInteger)  #Iht

washing = lp.LpVariable.dict("washing", (H, P, T), lowBound = 0, upBound = None, cat = lp.LpInteger)   #Whp

X = lp.LpVariable.dict("transport_mode", (H, H, P, M, T), cat = lp.LpBinary) #Xhh

recondition = lp.LpVariable.dict("reconditioned", (H, P, T),lowBound = 0, upBound = None, cat = lp.LpInteger) #Vhp

In [46]:
full_sea_dispatch = lp.LpVariable.dict("full_dispatch", (H, H, P, T), cat=lp.LpBinary)

## Parameters list:

In [47]:
#1
# p x i
# 3 x 6 = 18

#Weekly Demand:

container_requested_initial = {
    (1,1): 15, (1,2): 8, (1,3): 6, (1,4): 2, (1,5): 18, (1,6):6, (1,7):7,
    (2,1):0, (2,2): 2, (2,3): 0, (2,4): 9, (2,5): 0, (2,6):0, (2,7):0,
    (3,1):1, (3,2): 3, (3,3): 7, (3,4): 5, (3,5): 0, (3,6):0, (3,7):0
}

#Transform to monthly
factor = 4                 # weeks to month

container_requested = {k: v * factor for k, v in container_requested_initial.items()}
print(container_requested)


{(1, 1): 60, (1, 2): 32, (1, 3): 24, (1, 4): 8, (1, 5): 72, (1, 6): 24, (1, 7): 28, (2, 1): 0, (2, 2): 8, (2, 3): 0, (2, 4): 36, (2, 5): 0, (2, 6): 0, (2, 7): 0, (3, 1): 4, (3, 2): 12, (3, 3): 28, (3, 4): 20, (3, 5): 0, (3, 6): 0, (3, 7): 0}


In [48]:
# Plug this after your H, P, J, M, T definitions:
# Group hubs by continent:
europe_hubs = [1, 4, 6, 9]
na_hubs = [2, 3, 5, 8]
asia_hubs = [7]

In [49]:
# Create all intercontinental pairs (order matters for both directions)
intercontinental_pairs = []

# Europe <-> North America
for h in europe_hubs:
    for hh in na_hubs:
        intercontinental_pairs.append((h, hh))
        intercontinental_pairs.append((hh, h))
# Europe <-> Asia
for h in europe_hubs:
    for hh in asia_hubs:
        intercontinental_pairs.append((h, hh))
        intercontinental_pairs.append((hh, h))
# North America <-> Asia
for h in na_hubs:
    for hh in asia_hubs:
        intercontinental_pairs.append((h, hh))
        intercontinental_pairs.append((hh, h))


In [50]:
# Example: Mapping of customer to (pickup_hub, dropoff_hub)
customer_lane = {
    1: (2, 9),   # AIT
    2: (3, 3),   # Biocair
    3: (5, 7),   # DSV
    4: (8, 1),   # Expeditors
    5: (4, 4),   # Movianto
    6: (3, 3),   # Quick Int
    7: (4, 6)    # World Courier
}


In [51]:
#2 #3 #4

#h
storage_capacity  = {1: 383, 2: 329, 3: 393, 4:395, 5:318, 6:374, 7:370, 8:377, 9:397}

washing_capacity  = {1: 330, 2: 340, 3: 345, 4: 315, 5: 325 , 6:330 , 7:330 , 8: 335, 9:350 }

recondition_capacity  = {1:330, 2: 340, 3: 345, 4: 315, 5: 325 , 6:330 , 7:330 , 8: 335, 9: 350 }

In [52]:
#5

#h
#Inventory levels

Min_hp = {1:0,  2:0,  3:0,  4:0,  5:0,  6:0,  7:0,  8:0,  9:0}

In [53]:
#Min_hp = {1:23, 2:29, 3:33, 4:29, 5:18, 6:24, 7:20, 8:27, 9:47}
#Min_hp = {1:3, 2:9, 3:3, 4:9, 5:8, 6:4, 7:2, 8:7, 9:7}

In [54]:
#6
Max_hp = {1:343, 2:379, 3:353, 4:309, 5:318, 6:324, 7:320, 8:327, 9:347}

In [55]:
#7

#p tare weight per product p (assuming empty)


weight = {
    1:70,
    2:370,
    3:235.6
    }


In [56]:
#8

#p
#Sea freight capacity depending on a 40ft container
beta = {1:60 , 2:5 , 3:7}

#44 tonn van capacity per p

beta_1 = {1:66 , 2:8, 3:8}

In [57]:
#9


#m,p
# estimations are in KG of co2 per km, assuming the container is empty and plates
#Done per mode m only road, air & sea
#  m1 = 0.15193 m2= 0.64875 m3=0.00349
ef ={
    1: 0.15193,
    2: 0.64875,
    3: 0.00349
}

In [58]:
#12
#storage_cost(p,h) weekly? 0.5*7= 3.5 GBP
#h = 13
storage_cost = { 1: 3.5, 2: 3.5, 3: 3.5, 4: 3.5, 5: 3.5, 6: 3.5, 7: 3.5, 8: 3.5, 9: 3.5}

In [59]:
#12
#missing 3- PHC, 4- THE (TOWER owned hubs) and 8- SJU
#Assumed: Theale to be : 10 GBP
# AMS & SJU similar to BRU
# PHC similar to OAK


igi_cost = { 1: 23.82, 2: 11.91, 3: 12, 4: 10, 5: 11.91 , 6: 14.46, 7: 15.88, 8: 23.82, 9: 23.82}

In [60]:
#m,h,hh
new_dist = {
(1, 1, 1): 99999,
 (1, 1, 2): 99999,
 (1, 1, 3): 99999,
 (1, 1, 4): 489,
 (1, 1, 5): 99999,
 (1, 1, 6): 366,
 (1, 1, 7): 99999,
 (1, 1, 9): 224,
 (1, 1, 8): 99999,
 (1, 2, 1): 99999,
 (1, 2, 2): 99999,
 (1, 2, 3): 1255,
 (1, 2, 4): 99999,
 (1, 2, 5): 3218,
 (1, 2, 6): 99999,
 (1, 2, 7): 99999,
 (1, 2, 9): 99999,
 (1, 2, 8): 99999,
 (1, 3, 1): 99999,
 (1, 3, 2): 1255,
 (1, 3, 3): 99999,
 (1, 3, 4): 99999,
 (1, 3, 5): 4399,
 (1, 3, 6): 99999,
 (1, 3, 7): 99999,
 (1, 3, 9): 99999,
 (1, 3, 8): 99999,
 (1, 4, 1): 489,
 (1, 4, 2): 99999,
 (1, 4, 3): 99999,
 (1, 4, 4): 99999,
 (1, 4, 5): 99999,
 (1, 4, 6): 832,
 (1, 4, 7): 99999,
 (1, 4, 9): 611,
 (1, 4, 8): 99999,
 (1, 5, 1): 99999,
 (1, 5, 2): 3218,
 (1, 5, 3): 4399,
 (1, 5, 4): 99999,
 (1, 5, 5): 99999,
 (1, 5, 6): 99999,
 (1, 5, 7): 99999,
 (1, 5, 9): 99999,
 (1, 5, 8): 99999,
 (1, 6, 1): 366,
 (1, 6, 2): 99999,
 (1, 6, 3): 99999,
 (1, 6, 4): 832,
 (1, 6, 5): 99999,
 (1, 6, 6): 99999,
 (1, 6, 7): 99999,
 (1, 6, 9): 443,
 (1, 6, 8): 99999,
 (1, 7, 1): 99999,
 (1, 7, 2): 99999,
 (1, 7, 3): 99999,
 (1, 7, 4): 99999,
 (1, 7, 5): 99999,
 (1, 7, 6): 99999,
 (1, 7, 7): 99999,
 (1, 7, 9): 99999,
 (1, 7, 8): 99999,
 (1, 9, 1): 224,
 (1, 9, 2): 99999,
 (1, 9, 3): 99999,
 (1, 9, 4): 611,
 (1, 9, 5): 99999,
 (1, 9, 6): 443,
 (1, 9, 7): 99999,
 (1, 9, 9): 99999,
 (1, 9, 8): 99999,
 (1, 8, 1): 99999,
 (1, 8, 2): 99999,
 (1, 8, 3): 99999,
 (1, 8, 4): 99999,
 (1, 8, 5): 99999,
 (1, 8, 6): 99999,
 (1, 8, 7): 99999,
 (1, 8, 9): 99999,
 (1, 8, 8): 99999,
 (2, 1, 1): 99999,
 (2, 1, 2): 6761,
 (2, 1, 3): 6050,
 (2, 1, 4): 411,
 (2, 1, 5): 9100,
 (2, 1, 6): 281,
 (2, 1, 7): 9044,
 (2, 1, 9): 166,
 (2, 1, 8): 7090,
 (2, 2, 1): 6761,
 (2, 2, 2): 99999,
 (2, 2, 3): 1112,
 (2, 2, 4): 6371,
 (2, 2, 5): 2783,
 (2, 2, 6): 7031,
 (2, 2, 7): 11399,
 (2, 2, 9): 6672,
 (2, 2, 8): 3309,
 (2, 3, 1): 6050,
 (2, 3, 2): 1112,
 (2, 3, 3): 99999,
 (2, 3, 4): 5644,
 (2, 3, 5): 3877,
 (2, 3, 6): 6330,
 (2, 3, 7): 11942,
 (2, 3, 9): 5985,
 (2, 3, 8): 2543,
 (2, 4, 1): 411,
 (2, 4, 2): 6371,
 (2, 4, 3): 5644,
 (2, 4, 4): 99999,
 (2, 4, 5): 8754,
 (2, 4, 6): 692,
 (2, 4, 7): 9296,
 (2, 4, 9): 403,
 (2, 4, 8): 6694,
 (2, 5, 1): 9100,
 (2, 5, 2): 2783,
 (2, 5, 3): 3877,
 (2, 5, 4): 754,
 (2, 5, 5): 99999,
 (2, 5, 6): 9343,
 (2, 5, 7): 10448,
 (2, 5, 9): 8978,
 (2, 5, 8): 5443,
 (2, 6, 1): 281,
 (2, 6, 2): 7031,
 (2, 6, 3): 6330,
 (2, 6, 4): 692,
 (2, 6, 5): 9343,
 (2, 6, 6): 99999,
 (2, 6, 7): 8882,
 (2, 6, 9): 365,
 (2, 6, 8): 7358,
 (2, 7, 1): 9044,
 (2, 7, 2): 11399,
 (2, 7, 3): 11942,
 (2, 7, 4): 9296,
 (2, 7, 5): 10448,
 (2, 7, 6): 8882,
 (2, 7, 7): 99999,
 (2, 7, 9): 8942,
 (2, 7, 8): 14458,
 (2, 9, 1): 166,
 (2, 9, 2): 6672,
 (2, 9, 3): 5985,
 (2, 9, 4): 403,
 (2, 9, 5): 8978,
 (2, 9, 6): 365,
 (2, 9, 7): 8942,
 (2, 9, 9): 99999,
 (2, 9, 8): 7094,
 (2, 8, 1): 7090,
 (2, 8, 2): 3309,
 (2, 8, 3): 2543,
 (2, 8, 4): 6694,
 (2, 8, 5): 5443,
 (2, 8, 6): 7358,
 (2, 8, 7): 14458,
 (2, 8, 9): 7094,
 (2, 8, 8): 99999,
 (3, 1, 1): 99999,
 (3, 1, 2): 8163,
 (3, 1, 3): 6282,
 (3, 1, 4): 427,
 (3, 1, 5): 14453,
 (3, 1, 6): 575,
 (3, 1, 7): 19581,
 (3, 1, 9): 206,
 (3, 1, 8): 7144,
 (3, 2, 1): 8163,
 (3, 2, 2): 99999,
 (3, 2, 3): 5169,
 (3, 2, 4): 7789,
 (3, 2, 5): 13815,
 (3, 2, 6): 8338,
 (3, 2, 7): 24312,
 (3, 2, 9): 8238,
 (3, 2, 8): 7001,
 (3, 3, 1): 6282,
 (3, 3, 2): 5169,
 (3, 3, 3): 99999,
 (3, 3, 4): 5907,
 (3, 3, 5): 9095,
 (3, 3, 6): 6562,
 (3, 3, 7): 19592,
 (3, 3, 9): 6356,
 (3, 3, 8): 2471,
 (3, 4, 1): 427,
 (3, 4, 2): 7789,
 (3, 4, 3): 5907,
 (3, 4, 4): 99999,
 (3, 4, 5): 14079,
 (3, 4, 6): 848,
 (3, 4, 7): 19223,
 (3, 4, 9): 494,
 (3, 4, 8): 6769,
 (3, 5, 1): 14453,
 (3, 5, 2): 13815,
 (3, 5, 3): 9095,
 (3, 5, 4): 14079,
 (3, 5, 5): 99999,
 (3, 5, 6): 14881,
 (3, 5, 7): 10682,
 (3, 5, 9): 14528,
 (3, 5, 8): 7447,
 (3, 6, 1): 575,
 (3, 6, 2): 8338,
 (3, 6, 3): 6562,
 (3, 6, 4): 848,
 (3, 6, 5): 14881,
 (3, 6, 6): 99999,
 (3, 6, 7): 20009,
 (3, 6, 9): 399,
 (3, 6, 8): 7572,
 (3, 7, 1): 19581,
 (3, 7, 2): 24312,
 (3, 7, 3): 19592,
 (3, 7, 4): 19223,
 (3, 7, 5): 10682,
 (3, 7, 6): 20009,
 (3, 7, 7): 99999,
 (3, 7, 9): 19656,
 (3, 7, 8): 17944,
 (3, 9, 1): 206,
 (3, 9, 2): 8238,
 (3, 9, 3): 6356,
 (3, 9, 4): 494,
 (3, 9, 5): 14528,
 (3, 9, 6): 399,
 (3, 9, 7): 19656,
 (3, 9, 9): 99999,
 (3, 9, 8): 7218,
 (3, 8, 1): 7144,
 (3, 8, 2): 7001,
 (3, 8, 3): 2471,
 (3, 8, 4): 6769,
 (3, 8, 5): 7447,
 (3, 8, 6): 7572,
 (3, 8, 7): 17944,
 (3, 8, 9): 7218,
 (3, 8, 8): 99999}


In [61]:
#p,m,h,hh
# Hub mapping
hub_map = {
    "Brussels": 1,
    "Chicago": 2,
    "Penssauken": 3,
    "Theale": 4,
    "Los Angeles": 5,
    "Frankfurt": 6,
    "Shanghai": 7,
    "San Juan": 8,
    "Amsterdam": 9
}

# 9×9 matrices structured by (p, m)
# p  {1,2,3} = product type( 1= KT400, 2= KTM 32, 3 = KTM52)
# m  {1,2,3} = mode (1=Road, 2=Sea, 3=Air)

# ROAD (m=1)

road_kt400 = [
    [99999,99999,99999,   120,99999,  41.25,99999,99999,   25],     # Brussels
    [99999,99999,   114,99999,  174,99999,99999,99999,99999],       # Chicago
    [99999, 64.8, 99999,99999,  170,99999,99999,99999,99999],       # Penssauken
    [  120,99999,99999,99999,99999,    56,99999,99999, 29.75],      # Theale
    [99999,  313,   129,99999,99999,99999,99999,99999,99999],       # Los Angeles
    [   19,99999,99999,   45,99999,99999,99999,99999,   41],        # Frankfurt
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # Shanghai
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # San Juan
    [   25,99999,99999, 34.5,99999,   41,99999,99999,99999],        # Amsterdam
]

road_ktm32 = [
    [99999,99999,99999,   159,99999,   140,99999,99999,   140],     # Brussels
    [99999,99999,   203,99999,  307,99999,99999,99999,99999],       # Chicago
    [99999,   203,99999,99999,  347,99999,99999,99999,99999],       # Penssauken
    [  159,99999,99999,99999,99999,   159,99999,99999,   270],      # Theale
    [99999,  307,   347,99999,99999,99999,99999,99999,99999],       # Los Angeles
    [  140,99999,99999,   159,99999,99999,99999,99999,   140],      # Frankfurt
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # Shanghai
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # San Juan
    [  140,99999,99999,   270,99999,   140,99999,99999,99999],      # Amsterdam
]

road_ktm52 = [
    [99999,99999,99999,   223,99999,   231,99999,99999,   300],     # Brussels
    [99999,99999,   340,99999,  370,99999,99999,99999,99999],       # Chicago
    [99999,   340,99999,99999,  500,99999,99999,99999,99999],       # Penssauken
    [  223,99999,99999,99999,99999,   353,99999,99999,   320],      # Theale
    [99999,  370,   500,99999,99999,99999,99999,99999,99999],       # Los Angeles
    [  240,99999,99999,   350,99999,99999,99999,99999,   300],      # Frankfurt
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # Shanghai
    [99999,99999,99999,99999,99999,99999,99999,99999,99999],        # San Juan
    [  300,99999,99999,   320,99999,   300,99999,99999,99999],      # Amsterdam
]

# SEA (m=2)  -> 3 product types

sea_kt400 = [
    [99999,   92,  291,99999,   84,99999,  108,99999,99999],    # Brussels
    [   92,99999,99999,  200,99999,  200,  200,99999,   92],    # Chicago
    [  122,99999,99999,  122,99999,  122,  135,99999,  122],    # Penssauken
    [99999,  160,  147,99999,  160,99999,   67,99999,99999],    # Theale
    [  122,99999,99999,  122,99999,  122,  135,99999,  122],    # Los Angeles
    [99999,  122,  110,99999,  122,99999,  123,99999,99999],    # Frankfurt
    [  125,  133,  133,  125,  133,  125,99999,99999,  125],    # Shanghai
    [  500,  500,  500,  500,  500,  500,  500,99999,  500],    # San Juan
    [99999,   92,122.7,99999,   84,99999,  108,99999,99999],    # Amsterdam
]

sea_ktm32 = [
    [99999,  297,  267,99999,  500,99999,  610,  700,99999],    # Brussels
    [  297,99999,99999,  297,99999,  450,  625,  800,  800],    # Chicago
    [  267,99999,99999,  267,99999,  450,  659,  685,  800],    # Penssauken
    [99999,  297,  267,99999,  500,99999,  882,  610,99999],    # Theale
    [  500,99999,99999,  500,99999,  500,  612,  700,  500],    # Los Angeles
    [99999,  350,  350,99999,  550,99999,  650,  700,99999],    # Frankfurt
    [ 1000,  650,  659,  800,  700,  700,99999, 1000, 1000],    # Shanghai
    [  700,  800,  685,  610,  700,  700, 1000,99999, 1000],    # San Juan
    [99999,  800,  800,99999,  500,99999, 1000, 1000,99999],    # Amsterdam
]

sea_ktm52 = [
    [99999,  503,  641,99999,  650,99999, 1118,  859,99999],    # Brussels
    [  503,99999,99999,  465,99999,  650, 1400, 1231,  503],    # Chicago
    [  641,99999,99999,  616,99999,  650, 1400,  890,  650],    # Penssauken
    [99999,  465,  616,99999,  700,99999, 1164,  828,99999],    # Theale
    [  650,99999,99999,  700,99999,  750, 1054,  900,  700],    # Los Angeles
    [99999,  650,  650,99999,  750,99999, 1303,  860,99999],    # Frankfurt
    [ 1118, 1400, 1400, 1164, 1054, 1303,99999, 1300, 1200],    # Shanghai
    [  859, 1231,  890,  828,  900,  860, 1300,99999, 1200],    # San Juan
    [99999,  503,  650,99999,  700,99999, 1200, 1200,99999],    # Amsterdam
]

# AIR (m=3)  -> 3 product types

air_kt400 = [
    [99999, 1012.5, 1012.5, 803.2, 1012.5, 803.2, 1131.9, 1132.9, 1012.5],   # Brussels
    [1012.5, 99999, 803.2, 1012.5, 803.2, 1012.5, 1132.9, 1132.9, 1012.5],   # Chicago
    [1012.5, 803.2, 99999, 1012.5, 803.2, 1012.5, 1132.9, 1132.9, 1012.5],   # Penssauken
    [803.2, 1012.5, 1012.5, 99999, 1012.5, 803.2, 1132.9, 1132.9, 803.2],    # Theale
    [1012.5, 803.2, 803.2, 1012.5, 99999, 1012.5, 1132.9, 1132.9, 803.2],    # Los Angeles
    [803.2, 1012.5, 1012.5, 803.2, 1012.5, 99999, 1132.9, 1132.9, 803.2],    # Frankfurt
    [1131.9, 1132.9, 1132.9, 1132.9, 1132.9, 1132.9, 99999, 1132.9, 1132.9], # Shanghai
    [1132.9, 1132.9, 1132.9, 1132.9, 1132.9, 1132.9, 1132.9, 99999, 1132.9], # San Juan
    [1012.5, 1012.5, 1012.5, 803.2, 803.2, 803.2, 1132.9, 1132.9, 99999],    # Amsterdam
]


air_ktm32= [
    [99999, 1897.2, 1897.2, 1504.08, 1897.2, 1504.08, 2119.68, 2121.6, 1897.2],   # Brussels
    [1897.2, 99999, 1504.08, 1897.2, 1504.08, 1897.2, 2121.6, 2121.6, 1897.2],    # Chicago
    [1897.2, 1504.08, 99999, 1897.2, 1504.08, 1897.2, 2121.6, 2121.6, 1897.2],    # Penssauken
    [1504.08, 1897.2, 1897.2, 99999, 1897.2, 1504.08, 2121.6, 2121.6, 1504.08],   # Theale
    [1897.2, 1504.08, 1504.08, 1897.2, 99999, 1897.2, 2121.6, 2121.6, 1504.08],   # Los Angeles
    [1504.08, 1897.2, 1897.2, 1504.08, 1897.2, 99999, 2121.6, 2121.6, 1504.08],   # Frankfurt
    [2119.68, 2121.6, 2121.6, 2121.6, 2121.6, 2121.6, 99999, 2121.6, 2121.6],     # Shanghai
    [2121.6, 2121.6, 2121.6, 2121.6, 2121.6, 2121.6, 2121.6, 99999, 2121.6],      # San Juan
    [1897.2, 1897.2, 1897.2, 1504.08, 1504.08, 1504.08, 2121.6, 2121.6, 99999],   # Amsterdam
]

air_ktm52 = [
    [99999, 2635, 2635, 2089, 2635, 2089, 2944, 2948, 2635],   # Brussels
    [ 2635, 99999, 2089, 2635, 2089, 2635, 2948, 2948, 2635],   # Chicago
    [ 2635, 2089, 99999, 2635, 2089, 2635, 2948, 2948, 2635],   # Penssauken
    [ 2089, 2635, 2635, 99999, 2635, 2089, 2948, 2948, 2089],   # Theale
    [ 2635, 2089, 2089, 2635, 99999, 2635, 2948, 2948, 2089],   # Los Angeles
    [ 2089, 2635, 2635, 2089, 2635, 99999, 2948, 2948, 2089],   # Frankfurt
    [ 2944, 2948, 2948, 2948, 2948, 2948, 99999, 2948, 2948],   # Shanghai
    [ 2948, 2948, 2948, 2948, 2948, 2948, 2948, 99999, 2948],   # San Juan
    [ 2635, 2635, 2635, 2089, 2089, 2089, 2948, 2948, 99999],   # Amsterdam
]

# Single dictionary cost_matrices keyed by (p,m)
cost_matrices = {
    (1, 1): road_kt400,
    (2, 1): road_ktm32,
    (3, 1): road_ktm52,

    (1, 2): sea_kt400,
    (2, 2): sea_ktm32,
    (3, 2): sea_ktm52,

    (1, 3): air_kt400,
    (2, 3): air_ktm32,
    (3, 3): air_ktm52
}

# New dictionary with 4D keys: (p, m, from_hub, to_hub)
new_tc2 = {}

for p in [1, 2, 3]:            # product type
    for m in [1, 2, 3]:        # mode of transport
        matrix_9x9 = cost_matrices[(p, m)]
        for i in range(9):
            for j in range(9):
                from_hub = i + 1
                to_hub   = j + 1
                cost_val = matrix_9x9[i][j]
                new_tc2[(p, m, from_hub, to_hub)] = cost_val

# Print the fully updated dictionary
print("new_tc2 = {")
for key, value in new_tc2.items():
    print(f"    {key}: {value},")
print("}")


new_tc2 = {
    (1, 1, 1, 1): 99999,
    (1, 1, 1, 2): 99999,
    (1, 1, 1, 3): 99999,
    (1, 1, 1, 4): 120,
    (1, 1, 1, 5): 99999,
    (1, 1, 1, 6): 41.25,
    (1, 1, 1, 7): 99999,
    (1, 1, 1, 8): 99999,
    (1, 1, 1, 9): 25,
    (1, 1, 2, 1): 99999,
    (1, 1, 2, 2): 99999,
    (1, 1, 2, 3): 114,
    (1, 1, 2, 4): 99999,
    (1, 1, 2, 5): 174,
    (1, 1, 2, 6): 99999,
    (1, 1, 2, 7): 99999,
    (1, 1, 2, 8): 99999,
    (1, 1, 2, 9): 99999,
    (1, 1, 3, 1): 99999,
    (1, 1, 3, 2): 64.8,
    (1, 1, 3, 3): 99999,
    (1, 1, 3, 4): 99999,
    (1, 1, 3, 5): 170,
    (1, 1, 3, 6): 99999,
    (1, 1, 3, 7): 99999,
    (1, 1, 3, 8): 99999,
    (1, 1, 3, 9): 99999,
    (1, 1, 4, 1): 120,
    (1, 1, 4, 2): 99999,
    (1, 1, 4, 3): 99999,
    (1, 1, 4, 4): 99999,
    (1, 1, 4, 5): 99999,
    (1, 1, 4, 6): 56,
    (1, 1, 4, 7): 99999,
    (1, 1, 4, 8): 99999,
    (1, 1, 4, 9): 29.75,
    (1, 1, 5, 1): 99999,
    (1, 1, 5, 2): 313,
    (1, 1, 5, 3): 129,
    (1, 1, 5, 4): 99999,
    (1, 1

In [62]:
#Cost of 40ft container from hub to hub
#(h,h)
container_cost_40ft = {
    (1, 1): 99999, (1, 2): 5000,  (1, 3): 4477, (1, 4): 99999, (1, 5): 5000,  (1, 6): 99999, (1, 7): 4000,  (1, 8): 5298,  (1, 9): 99999,
    (2, 1): 2409,  (2, 2): 99999, (2, 3): 99999, (2, 4): 2279,  (2, 5): 99999, (2, 6): 2400,  (2, 7): 4000,  (2, 8): 5973,  (2, 9): 2400,
    (3, 1): 2000,  (3, 2): 99999, (3, 3): 99999, (3, 4): 1812,  (3, 5): 99999, (3, 6): 2000,  (3, 7): 4000,  (3, 8): 4332, (3, 9): 2000,
    (4, 1): 99999, (4, 2): 4000,  (4, 3): 3391, (4, 4): 99999, (4, 5): 4000,  (4, 6): 99999, (4, 7): 2590,  (4, 8): 4143,  (4, 9): 99999,
    (5, 1): 3000,  (5, 2): 99999, (5, 3): 99999, (5, 4): 2936,  (5, 5): 99999, (5, 6): 3000,  (5, 7): 2275,  (5, 8): 6000,  (5, 9): 3000,
    (6, 1): 99999, (6, 2): 4600,  (6, 3): 3955.43,  (6, 4): 99999, (6, 5): 4000,  (6, 6): 99999, (6, 7): 4000,  (6, 8): 4937,  (6, 9): 99999,
    (7, 1): 6500,  (7, 2): 4223.45,  (7, 3): 5308,  (7, 4): 6289,  (7, 5): 4210,  (7, 6): 6500,  (7, 7): 99999, (7, 8): 6000,  (7, 9): 6500,
    (8, 1): 7000,  (8, 2): 7000,  (8, 3): 7000,  (8, 4): 7000,  (8, 5): 7000,  (8, 6): 7000,  (8, 7): 7000,  (8, 8): 99999, (8, 9): 7000,
    (9, 1): 99999,  (9, 2): 4446,  (9, 3): 3943,  (9, 4): 99999,  (9, 5): 4000,  (9, 6): 99999,  (9, 7): 4000,  (9, 8): 6000,  (9, 9): 99999,
}

In [63]:
container_cost_van = {
    (1, 1): 99999, (1, 2): 99999, (1, 3): 99999, (1, 4): 1570,   (1, 5): 99999,
    (1, 6): 1600,  (1, 7): 99999, (1, 8): 99999, (1, 9): 1600,

    (2, 1): 99999, (2, 2): 99999, (2, 3): 1630,  (2, 4): 99999,  (2, 5): 2700,
    (2, 6): 99999, (2, 7): 99999, (2, 8): 99999, (2, 9): 99999,

    (3, 1): 99999, (3, 2): 2038.93, (3, 3): 99999, (3, 4): 99999, (3, 5): 2963,
    (3, 6): 99999, (3, 7): 99999,  (3, 8): 99999, (3, 9): 99999,

    (4, 1): 1570,  (4, 2): 99999, (4, 3): 99999, (4, 4): 99999,  (4, 5): 99999,
    (4, 6): 1600,  (4, 7): 99999, (4, 8): 99999, (4, 9): 1400,

    (5, 1): 99999, (5, 2): 2963,  (5, 3): 4223,  (5, 4): 99999,  (5, 5): 99999,
    (5, 6): 99999, (5, 7): 99999, (5, 8): 99999, (5, 9): 99999,

    (6, 1): 1600,  (6, 2): 99999, (6, 3): 99999, (6, 4): 2074,   (6, 5): 99999,
    (6, 6): 99999, (6, 7): 99999, (6, 8): 99999, (6, 9): 1700,

    (7, 1): 99999, (7, 2): 99999, (7, 3): 99999, (7, 4): 99999,  (7, 5): 99999,
    (7, 6): 99999, (7, 7): 99999, (7, 8): 99999, (7, 9): 99999,

    (8, 1): 99999, (8, 2): 99999, (8, 3): 99999, (8, 4): 99999,  (8, 5): 99999,
    (8, 6): 99999, (8, 7): 99999, (8, 8): 99999, (8, 9): 99999,

    (9, 1): 1600,  (9, 2): 99999, (9, 3): 99999, (9, 4): 1740,   (9, 5): 99999,
    (9, 6): 1700,  (9, 7): 99999, (9, 8): 99999, (9, 9): 99999,
}


In [64]:
#m,h,hh
#road, air, sea
new_time = {
 (1, 1, 1): 99999,
 (1, 1, 2): 99999.0,
 (1, 1, 3): 99999.0,
 (1, 1, 4): 6.11,
 (1, 1, 5): 99999.0,
 (1, 1, 6): 4.58,
 (1, 1, 7): 99999.0,
 (1, 1, 9): 2.8,
 (1, 1, 8): 99999.0,
 (1, 2, 1): 99999.0,
 (1, 2, 2): 99999,
 (1, 2, 3): 15.69,
 (1, 2, 4): 99999.0,
 (1, 2, 5): 40.23,
 (1, 2, 6): 99999.0,
 (1, 2, 7): 99999.0,
 (1, 2, 9): 99999.0,
 (1, 2, 8): 99999.0,
 (1, 3, 1): 99999.0,
 (1, 3, 2): 15.69,
 (1, 3, 3): 99999,
 (1, 3, 4): 99999.0,
 (1, 3, 5): 54.99,
 (1, 3, 6): 99999.0,
 (1, 3, 7): 99999.0,
 (1, 3, 9): 99999.0,
 (1, 3, 8): 99999.0,
 (1, 4, 1): 6.11,
 (1, 4, 2): 99999.0,
 (1, 4, 3): 99999.0,
 (1, 4, 4): 99999,
 (1, 4, 5): 99999.0,
 (1, 4, 6): 10.4,
 (1, 4, 7): 99999.0,
 (1, 4, 9): 7.64,
 (1, 4, 8): 99999.0,
 (1, 5, 1): 99999.0,
 (1, 5, 2): 40.23,
 (1, 5, 3): 54.99,
 (1, 5, 4): 99999.0,
 (1, 5, 5): 99999,
 (1, 5, 6): 99999.0,
 (1, 5, 7): 99999.0,
 (1, 5, 9): 99999.0,
 (1, 5, 8): 99999.0,
 (1, 6, 1): 4.58,
 (1, 6, 2): 99999.0,
 (1, 6, 3): 99999.0,
 (1, 6, 4): 10.4,
 (1, 6, 5): 99999.0,
 (1, 6, 6): 99999,
 (1, 6, 7): 99999.0,
 (1, 6, 9): 5.54,
 (1, 6, 8): 99999.0,
 (1, 7, 1): 99999.0,
 (1, 7, 2): 99999.0,
 (1, 7, 3): 99999.0,
 (1, 7, 4): 99999.0,
 (1, 7, 5): 99999.0,
 (1, 7, 6): 99999.0,
 (1, 7, 7): 99999,
 (1, 7, 9): 99999.0,
 (1, 7, 8): 99999.0,
 (1, 9, 1): 2.8,
 (1, 9, 2): 99999.0,
 (1, 9, 3): 99999.0,
 (1, 9, 4): 7.64,
 (1, 9, 5): 99999.0,
 (1, 9, 6): 5.54,
 (1, 9, 7): 99999.0,
 (1, 9, 9): 99999.0,
 (1, 9, 8): 99999.0,
 (1, 8, 1): 99999.0,
 (1, 8, 2): 99999.0,
 (1, 8, 3): 99999.0,
 (1, 8, 4): 99999.0,
 (1, 8, 5): 99999.0,
 (1, 8, 6): 99999.0,
 (1, 8, 7): 99999.0,
 (1, 8, 9): 99999.0,
 (1, 8, 8): 99999.0,

    (2, 1, 2): 9.95, (2, 1, 3): 9.06, (2, 1, 4): 2.01, (2, 1, 5): 12.88,
    (2, 1, 6): 1.85, (2, 1, 7): 12.81, (2, 1, 9): 1.71, (2, 1, 8): 10.36,
    (2, 2, 1): 9.95, (2, 2, 2): 99999, (2, 2, 3): 2.89, (2, 2, 4): 9.46,
    (2, 2, 5): 4.98, (2, 2, 6): 10.29, (2, 2, 7): 15.75, (2, 2, 9): 9.84,
    (2, 2, 8): 5.64, (2, 3, 1): 9.06, (2, 3, 2): 2.89, (2, 3, 3): 99999,
    (2, 3, 4): 8.56, (2, 3, 5): 6.35, (2, 3, 6): 9.41, (2, 3, 7): 16.43,
    (2, 3, 9): 8.98, (2, 3, 8): 4.68, (2, 4, 1): 2.01, (2, 4, 2): 9.46,
    (2, 4, 3): 8.56, (2, 4, 4): 99999, (2, 4, 5): 12.44, (2, 4, 6): 2.37,
    (2, 4, 7): 13.12, (2, 4, 9): 2.0, (2, 4, 8): 9.87, (2, 5, 1): 12.88,
    (2, 5, 2): 4.98, (2, 5, 3): 6.35, (2, 5, 4): 10.93, (2, 5, 5): 99999,
    (2, 5, 6): 13.18, (2, 5, 7): 14.56, (2, 5, 9): 12.72, (2, 5, 8): 8.3,
    (2, 6, 1): 1.85, (2, 6, 2): 10.29, (2, 6, 3): 9.41, (2, 6, 4): 2.37,
    (2, 6, 5): 13.18, (2, 6, 6): 99999, (2, 6, 7): 12.6, (2, 6, 9): 1.96,
    (2, 6, 8): 10.7, (2, 7, 1): 12.81, (2, 7, 2): 15.75, (2, 7, 3): 16.43,
    (2, 7, 4): 13.12, (2, 7, 5): 14.56, (2, 7, 6): 12.6, (2, 7, 7): 99999,
    (2, 7, 9): 12.68, (2, 7, 8): 19.57, (2, 9, 1): 1.71, (2, 9, 2): 9.84,
    (2, 9, 3): 8.98, (2, 9, 4): 2.0, (2, 9, 5): 12.72, (2, 9, 6): 1.96,
    (2, 9, 7): 12.68, (2, 9, 9): 99999, (2, 9, 8): 10.37, (2, 8, 1): 10.36,
    (2, 8, 2): 5.64, (2, 8, 3): 4.68, (2, 8, 4): 9.87, (2, 8, 5): 8.3,
    (2, 8, 6): 10.7, (2, 8, 7): 19.57, (2, 8, 9): 10.37,
 (2, 8, 8): 99999,
 (3, 1, 1): 99999,
 (3, 1, 2): 220.62,
 (3, 1, 3): 169.78,
 (3, 1, 4): 11.54,
 (3, 1, 5): 390.89,
 (3, 1, 6): 15.54,
 (3, 1, 7): 529.21,
 (3, 1, 9): 5.57,
 (3, 1, 8): 193.08,
 (3, 2, 1): 220.62,
 (3, 2, 2): 99999,
 (3, 2, 3): 139.7,
 (3, 2, 4): 210.51,
 (3, 2, 5): 373.38,
 (3, 2, 6): 225.35,
 (3, 2, 7): 656.0,
 (3, 2, 9): 222.65,
 (3, 2, 8): 189.21,
 (3, 3, 1): 169.78,
 (3, 3, 2): 139.7,
 (3, 3, 3): 99999,
 (3, 3, 4): 159.11,
 (3, 3, 5): 245.81,
 (3, 3, 6): 177.89,
 (3, 3, 7): 529.51,
 (3, 3, 9): 171.78,
 (3, 3, 8): 66.78,
 (3, 4, 1): 11.54,
 (3, 4, 2): 210.51,
 (3, 4, 3): 159.11,
 (3, 4, 4): 99999,
 (3, 4, 5): 380.51,
 (3, 4, 6): 22.92,
 (3, 4, 7): 519.54,
 (3, 4, 9): 13.35,
 (3, 4, 8): 182.94,
 (3, 5, 1): 390.89,
 (3, 5, 2): 373.38,
 (3, 5, 3): 245.81,
 (3, 5, 4): 380.51,
 (3, 5, 5): 99999,
 (3, 5, 6): 402.19,
 (3, 5, 7): 288.16,
 (3, 5, 9): 392.65,
 (3, 5, 8): 201.24,
 (3, 6, 1): 15.54,
 (3, 6, 2): 225.35,
 (3, 6, 3): 177.89,
 (3, 6, 4): 22.92,
 (3, 6, 5): 402.19,
 (3, 6, 6): 99999,
 (3, 6, 7): 540.76,
 (3, 6, 9): 10.78,
 (3, 6, 8): 204.65,
 (3, 7, 1): 529.21,
 (3, 7, 2): 656.0,
 (3, 7, 3): 529.51,
 (3, 7, 4): 519.54,
 (3, 7, 5): 288.16,
 (3, 7, 6): 540.76,
 (3, 7, 7): 99999,
 (3, 7, 9): 531.24,
 (3, 7, 8): 485.24,
 (3, 9, 1): 5.57,
 (3, 9, 2): 222.65,
 (3, 9, 3): 171.78,
 (3, 9, 4): 13.35,
 (3, 9, 5): 392.65,
 (3, 9, 6): 10.78,
 (3, 9, 7): 531.24,
 (3, 9, 9): 99999,
 (3, 9, 8): 195.62,
 (3, 8, 1): 193.08,
 (3, 8, 2): 189.21,
 (3, 8, 3): 66.78,
 (3, 8, 4): 182.94,
 (3, 8, 5): 201.24,
 (3, 8, 6): 204.65,
 (3, 8, 7): 485.24,
 (3, 8, 9): 195.62,
 (3, 8, 8): 99999
}




In [65]:
# Multiply every value (that is not 99999 or 99999.0) by 0.00595238.
#for key, value in new_time.items():
#    if value != 99999 and value != 99999.0:
#        new_time[key] = value * 0.00595238

# Print the updated dictionary.
#print(new_time)

## Objective Functions:






1.   Min cost
2.   Min lead time
3.   Min carbon emissions


---


1. Min cost (movement + inventory + cleaning + reconditioning)

In [66]:
Inventory_cost = lp.lpSum(storage_cost.get((h)) * inventory[h, p, j, t] for h in H for t in T for p in P for j in J)

In [67]:
Cleaning_cost = lp.lpSum(igi_cost.get((h)) * recondition[h, p, t] for h in H for p in P for t in T)

In [68]:
Washing_cost = lp.lpSum(igi_cost.get((h)) * washing[h, p, t] for h in H for p in P for t in T)

Movement costs

In [69]:
Moving_road_cost = lp.lpSum(container_cost_van.get(((h,hh))) * (move[h,hh,p,j,1,t] * (1 / beta_1.get(p))) for h in H for hh in H if hh != h for p in P for j in J for t in T)

In [70]:
Moving_air_cost = lp.lpSum(new_tc2.get(((p,3,h,hh))) * (move[h,hh,p,j,2,t])  for h in H for hh in H if hh != h for p in P for j in J for t in T)

In [71]:
Moving_sea_cost = lp.lpSum(container_cost_40ft.get(((h,hh))) * (move[h,hh,p,j,3,t] * ( 1 / beta.get(p)))  for h in H for hh in H if hh != h for p in P for j in J for t in T )

In [72]:
#(1)
Logistics_cost =  Moving_road_cost + Moving_air_cost + Moving_sea_cost + Inventory_cost + Cleaning_cost + Washing_cost

2. Min Lead Time (Delivery time from hub to delivery points + Delivery time from collection points to hubs + Delivery time from one hub to another hub by different transportation modes]  

In [73]:
transit_time_sea = lp.lpSum(new_time.get((3,h,hh)) * (full_sea_dispatch[h,hh,p,t]) for h in H for hh in H if hh != h for p in P  for t in T )

In [74]:
transit_time_road = lp.lpSum(new_time.get((1,h,hh)) * (full_sea_dispatch[h,hh,p,t]) for h in H for hh in H if hh != h for p in P for t in T )

In [75]:
transit_time_air = lp.lpSum(new_time.get((2,h,hh)) * move[h,hh,p,j,2,t] for h in H for hh in H if hh !=h for p in P for j in J for t in T)

In [76]:
transit_time = transit_time_sea + transit_time_road + transit_time_air

3. Min Carbon Footprint (Emission from hub to delivery points + Emission from collection points to hubs + Emission from one hub to another hub by different mode of transportation]   

In [77]:
#(3)
carbon_emissions = lp.lpSum((new_dist.get((m,h,hh)) * weight.get(p) * ef.get(m) * move[h,hh,p,j,m,t])* (1/1000) for h in H for hh in H if hh != h for m in M for t in T for p in P for j in J)

## Constraints

In [78]:
def build_base_model():
    model = lp.LpProblem("TransportationModel", lp.LpMinimize)

    # CUSTOMER DEMAND SATISFACTION CONSTRAINTS
    for i in I:
        pickup_hub, dropoff_hub = customer_lane[i]  # defined in data parameters
        for p in P:
            # Delivery constraint
            model += lp.lpSum(deliver[pickup_hub, i, p, t] for t in T) == container_requested[(p, i)], \
                     f"Demand_delivery_customer_{i}_prod_{p}"
            # Collection constraint
            model += lp.lpSum(collect[i, dropoff_hub, p, t] for t in T) == container_requested[(p, i)], \
                     f"Demand_collection_customer_{i}_prod_{p}"

    # INVENTORY BALANCE CONSTRAINTS
    # (a) Cleaned Inventory (state 2)
    for h in H:
        for p in P:
            for t in T:
                if t == T[0]:
                    initial_clean = 0  # set an initial value
                    model += inventory[h, p, 2, t] == initial_clean \
                        + washing[h, p, t] + recondition[h, p, t] \
                        + lp.lpSum(move[hh, h, p, 2, m, t] for hh in H if hh != h for m in M) \
                        - lp.lpSum(deliver[h, i, p, t] for i in I if customer_lane[i][0] == h) \
                        - lp.lpSum(move[h, hh, p, 2, m, t] for hh in H if hh != h for m in M), \
                        f"Clean_inventory_balance_h{h}_p{p}_t{t}"
                else:
                    model += inventory[h, p, 2, t] == inventory[h, p, 2, t-1] \
                        + washing[h, p, t] + recondition[h, p, t] \
                        + lp.lpSum(move[hh, h, p, 2, m, t] for hh in H if hh != h for m in M) \
                        - lp.lpSum(deliver[h, i, p, t] for i in I if customer_lane[i][0] == h) \
                        - lp.lpSum(move[h, hh, p, 2, m, t] for hh in H if hh != h for m in M), \
                        f"Clean_inventory_balance_h{h}_p{p}_t{t}"

    # (b) Used/unclean Inventory (state 1)
    for h in H:
        for p in P:
            for t in T:
                if t == T[0]:
                    initial_dirty = 0
                    model += inventory[h, p, 1, t] == initial_dirty \
                        + lp.lpSum(collect[i, h, p, t] for i in I if customer_lane[i][1] == h) \
                        - washing[h, p, t] - recondition[h, p, t], \
                        f"Dirty_inventory_balance_h{h}_p{p}_t{t}"
                else:
                    model += inventory[h, p, 1, t] == inventory[h, p, 1, t-1] \
                        + lp.lpSum(collect[i, h, p, t] for i in I if customer_lane[i][1] == h) \
                        - washing[h, p, t] - recondition[h, p, t], \
                        f"Dirty_inventory_balance_h{h}_p{p}_t{t}"

    #  STORAGE, WASHING, AND RECONDITIONING CAPACITY CONSTRAINTS
    for h in H:
        for p in P:
            for t in T:
                for j in J:
                    model += inventory[h, p, j, t] <= storage_capacity.get(h, 0), \
                             f"Storage_capacity_h{h}_p{p}_state{j}_t{t}"
                    model += inventory[h, p, j, t] >= Min_hp.get(h, 0), \
                             f"Min_inventory_h{h}_p{p}_state{j}_t{t}"

    for h in H:
        for p in P:
            for t in T:
                model += washing[h, p, t] <= washing_capacity.get(h, 0), \
                         f"Washing_capacity_h{h}_p{p}_t{t}"

    for h in H:
        for p in P:
            for t in T:
                model += recondition[h, p, t] <= recondition_capacity.get(h, 0), \
                         f"Recondition_capacity_h{h}_p{p}_t{t}"

    # MOVEMENT RESTRICTIONS
    for h in H:
        for p in P:
            for m in M:
                for t in T:
                    model += move[h, h, p, 2, m, t] == 0, f"No_self_move_clean_h{h}_p{p}_m{m}_t{t}"
                    model += move[h, h, p, 1, m, t] == 0, f"No_self_move_dirty_h{h}_p{p}_m{m}_t{t}"

    for h in H:
        for hh in H:
            if hh != h:
                for p in P:
                    for m in M:
                        for t in T:
                            model += move[h, hh, p, 1, m, t] == 0, f"No_dirty_move_from_{h}_to_{hh}_p{p}_m{m}_t{t}"





    # Full-load enforcement for clean container sea freight movements (mode 3)
    for h in H:
        for hh in H:
            if h != hh:
                for p in P:
                    for t in T:
                        full_capacity = beta[p]  # full container load for this product type
                        var = move[h, hh, p, 2, 2, t]  # j = 2 (clean), m = 3 (Sea freight)
                        act = full_sea_dispatch[h, hh, p,t]

                        # 1. If movement happens, enforce at least one full container
                        model += var >= full_capacity * act, \
                          f"MinFullSea_h{h}_to_{hh}_p{p}_t{t}"

                    # 2. If no movement, ensure value is 0
                        model += var <= full_capacity * act, \
                          f"MaxFullSea_h{h}_to_{hh}_p{p}_t{t}"



    # Full-load enforcement for clean container road freight movements (mode = 1)
    for h in H:
        for hh in H:
            if h != hh:
                for p in P:
                    for t in T:
                        full_capacity1 = beta_1[p]  # full container load for this product type
                        var1 = move[h, hh, p, 2, 1, t]  # j = 2 (clean), m = 1 (Road freight)
                        act1 = full_sea_dispatch[h, hh, p,t]

                        # 1. If movement happens, enforce at least one full container
                        model += var1 >= full_capacity1 * act1 * 0.70, \
                          f"MinFullRoad_h{h}_to_{hh}_p{p}_t{t}"

                    # 2. If no movement, ensure value is 0
                        model += var1 <= full_capacity1 * act1, \
                          f"MaxFullRoad_h{h}_to_{hh}_p{p}_t{t}"

     # Forbid road freight (mode 1) on intercontinental pairs
    #for (h, hh) in intercontinental_pairs:
       #for p in P:
            #for j in J:
                #for t in T:
                    #model += move[h, hh, p, j, 1, t] == 0, f"NoRoadIntercont_{h}_{hh}_p{p}_j{j}_t{t}"

    return model

## Execution

In [79]:
import pandas as pd
import pulp as lp
from itertools import product

HUBS = {1: "Brussels", 2: "Chicago", 3: "Pennsauken", 4: "Theale", 5: "Los Angeles", 6: "Frankfurt", 7: "Shanghai", 8: "San Juan", 9: "Amsterdam"}
PRODUCTS = {1: "KT 400", 2: "KTM 52", 3: "KTM 32"}
STATES = {1: "Clean", 2: "Dirty"}
MODES = {1: "Road", 2: "Air", 3: "Sea"}

def moves_dict_to_df(decision_vars):
    records = []
    for key, qty in decision_vars.items():
        if qty <= 0 or not key.startswith("moved_"):
            continue
        parts = key.split("_")
        if len(parts) != 7:
            continue
        h1, h2, p, j, m, t = map(int, parts[1:])
        records.append({
            "Origin Hub": HUBS.get(h1, h1),
            "Destination Hub": HUBS.get(h2, h2),
            "Product": PRODUCTS.get(p, p),
            "State": STATES.get(j, j),
            "Transport Mode": MODES.get(m, m),
            "Month": t,
            "Quantity": qty,
        })
    return pd.DataFrame.from_records(records)

# 10 grid points for weights: e.g. varying cost from 0.1 to 0.9 (with fixed sum 1)
cost_weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.0]
carbon_weights = [1.0 - c - 0.2 for c in cost_weights]  # e.g. time fixed at 0.2, rest to carbon
time_weight = 0.2

summary = []
move_dfs = {}

for idx, (w_cost, w_carbon) in enumerate(zip(cost_weights, carbon_weights), 1):
    model = build_base_model()
    # Define objectives ONCE for this model instance
    Inventory_cost = lp.lpSum(storage_cost.get((h)) * inventory[h, p, j, t] for h in H for t in T for p in P for j in J)
    Cleaning_cost = lp.lpSum(igi_cost.get((h)) * recondition[h, p, t] for h in H for p in P for t in T)
    Washing_cost = lp.lpSum(igi_cost.get((h)) * washing[h, p, t] for h in H for p in P for t in T)
    Moving_road_cost = lp.lpSum(container_cost_van.get(((h,hh))) * (move[h,hh,p,j,1,t] * (1 / beta_1.get(p)))
                                for h in H for hh in H if hh != h for p in P for j in J for t in T)
    Moving_air_cost = lp.lpSum(new_tc2.get(((p,3,h,hh))) * (move[h,hh,p,j,2,t])
                               for h in H for hh in H if hh != h for p in P for j in J for t in T)
    Moving_sea_cost = lp.lpSum(container_cost_40ft.get(((h,hh))) * (move[h,hh,p,j,3,t] * (1 / beta.get(p)))
                               for h in H for hh in H if hh != h for p in P for j in J for t in T)
    logistics_cost = Moving_road_cost + Moving_air_cost + Moving_sea_cost + Inventory_cost + Cleaning_cost + Washing_cost

    transit_time_sea = lp.lpSum(new_time.get((3,h,hh)) * (full_sea_dispatch[h,hh,p,t])
                                for h in H for hh in H if hh != h for p in P for t in T)
    transit_time_road = lp.lpSum(new_time.get((1,h,hh)) * (full_sea_dispatch[h,hh,p,t])
                                 for h in H for hh in H if hh != h for p in P for t in T)
    transit_time_air = lp.lpSum(new_time.get((2,h,hh)) * move[h,hh,p,j,2,t]
                                for h in H for hh in H if hh != h for p in P for j in J for t in T)
    transit_time = transit_time_sea + transit_time_road + transit_time_air

    carbon_emissions = lp.lpSum((new_dist.get((m,h,hh)) * weight.get(p) * ef.get(m) * move[h,hh,p,j,m,t]) * (1/1000)
                                for h in H for hh in H if hh != h for m in M for t in T for p in P for j in J)

    model.objective = None
    model += w_cost * logistics_cost + w_carbon * carbon_emissions + time_weight * transit_time

    model.solve(lp.PULP_CBC_CMD(msg=1))
    log_cost = lp.value(logistics_cost)
    log_carb = lp.value(carbon_emissions)
    log_time = lp.value(transit_time)
    print(f"Grid {idx}: CostW={w_cost:.2f}, CarbW={w_carbon:.2f}, TimeW={time_weight:.2f} | Cost={log_cost:.1f} Carb={log_carb:.1f} Time={log_time:.1f} | Status: {lp.LpStatus[model.status]}")

    summary.append({
        "Grid_ID": idx,
        "Cost_weight": w_cost,
        "Carbon_weight": w_carbon,
        "Time_weight": time_weight,
        "Logistics_Cost": log_cost,
        "Carbon_Emissions": log_carb,
        "Transit_Time": log_time,
        "Status": lp.LpStatus[model.status]
    })
    moves_vars = {v.name: v.varValue for v in model.variables() if v.varValue and v.name.startswith("moved_")}
    move_dfs[f"Solution_{idx}"] = moves_dict_to_df(moves_vars)
    move_dfs[f"Solution_{idx}"]["Grid_ID"] = idx

# Export summary
pd.DataFrame(summary).to_csv("filtered_weighted_sum.csv", index=False)

# Export Excel with each solution as a tab
with pd.ExcelWriter("all_moves_weighted_sum.xlsx") as writer:
    for sol_name, df in move_dfs.items():
        df.to_excel(writer, sheet_name=sol_name, index=False)
print("✅ Exported: filtered_weighted_sum.csv and all_moves_weighted_sum.xlsx (10 grid points, 10 tabs)")


Grid 1: CostW=0.10, CarbW=0.70, TimeW=0.20 | Cost=88102.8 Carb=1005.2 Time=0.0 | Status: Optimal
Grid 2: CostW=0.20, CarbW=0.60, TimeW=0.20 | Cost=87855.6 Carb=1070.8 Time=0.0 | Status: Optimal
Grid 3: CostW=0.30, CarbW=0.50, TimeW=0.20 | Cost=87855.6 Carb=1070.8 Time=0.0 | Status: Optimal
Grid 4: CostW=0.40, CarbW=0.40, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 5: CostW=0.50, CarbW=0.30, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 6: CostW=0.60, CarbW=0.20, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 7: CostW=0.70, CarbW=0.10, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 8: CostW=0.80, CarbW=-0.00, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 9: CostW=0.90, CarbW=-0.10, TimeW=0.20 | Cost=87677.6 Carb=1188.4 Time=0.0 | Status: Optimal
Grid 10: CostW=0.00, CarbW=0.80, TimeW=0.20 | Cost=135694.6 Carb=924.0 Time=0.0 | Status: Optimal
✅ Exported: filtered_weight